'-1/18*i*e^2*gamma_{%\\rho_52180,%eta_21888,%eps_14253}*gamma_{+%\\rho_52180,%gam_17939,%del_15647}*d_{j_18804,%gam_17939}(p_4)_u^(*)*d_{i_18804,%del_15647}(p_3)_v*s_{k_18802,%eps_14253}(p_1)_u*s_{l_18802,%eta_21888}(p_2)_v^(*)/(m_s^2 + s_12 + 1/2*reg_prop)' 

'-1/18*i*e^2*gamma_{\\INDEX_0,INDEX_1,INDEX_2}*gamma_{+\\INDEX_0,INDEX_3,INDEX_4}*d_{MOMENTUM_0,INDEX_3}(p_4)_u^(*)*d_{MOMENTUM_1,INDEX_4}(p_3)_v*s_{MOMENTUM_2,INDEX_2}(p_1)_u*s_{MOMENTUM_3,INDEX_1}(p_2)_v^(*)/(m_s^2+s_12+1/2*reg_prop)'

['-', '1/18', '*', 'i', '*', 'e', '^', '2', '*', 'gamma_{', '\\', 'INDEX_0', ',', 'INDEX_1', ',', 'INDEX_2', '}', '*', 'gamma_{', '+', '\\', 'INDEX_0', ',', 'INDEX_3', ',', 'INDEX_4', '}', '*', 'd_{', 'MOMENTUM_0', ',', 'INDEX_3', '}', '(', 'p_4', ')', '_u', '^', '(', '*', ')', '*', 'd_{', 'MOMENTUM_1', ',', 'INDEX_4', '}', '(', 'p_3', ')', '_v', '*', 's_{', 'MOMENTUM_2', ',', 'INDEX_2', '}', '(', 'p_1', ')', '_u', '*', 's_{', 'MOMENTUM_3', ',', 'INDEX_1', '}', '(', 'p_2', ')', '_v', '^', '(', '*', ')', '/', '(', 'm_s', '^', '2', '+', 's_12', '+', '1/2', '*', 'reg_prop', ')']


'1/324*e^4*(16*m_d^2*m_s^2 + 8*m_d^2*s_12 + 8*s_14*s_23 + 8*s_13*s_24 + 8*m_s^2*s_34)*(m_s^2 + s_12 + 1/2*reg_prop)^(-2)'

['1/324', '*', 'e', '^', '4', '*', '(', '16', '*', 'm_d', '^', '2', '*', 'm_s', '^', '2', '+', '8', '*', 'm_d', '^', '2', '*', 's_12', '+', '8', '*', 's_14', '*', 's_23', '+', '8', '*', 's_13', '*', 's_24', '+', '8', '*', 'm_s', '^', '2', '*', 's_34', ')', '*', '(', 'm_s', '^', '2', '+', 's_12', '+', '1/2', '*', 'reg_prop', ')', '^', '(', '-', '2', ')']


In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import json

from sklearn.model_selection import train_test_split
from collections import Counter, OrderedDict
from itertools import cycle
import re
import random
from torchtext.vocab import vocab
from tqdm import tqdm
import warnings

/global/homes/r/ritesh11/.local/perlmutter/pytorch2.3.1/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/global/homes/r/ritesh11/.local/perlmutter/pytorch2.3.1/lib/python3.11/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [2]:
class Tokenizer:
    """
    Tokenizer for processing symbolic mathematical expressions.
    """
    def __init__(self, df, index_token_pool_size, momentum_token_pool_size, special_symbols, UNK_IDX, to_replace):
        self.amps = df.amp.tolist()
        self.sqamps = df.sqamp.tolist()

        # Issue warnings if token pool sizes are too small
        if index_token_pool_size < 100:
            warnings.warn(f"Index token pool size ({index_token_pool_size}) is small. Consider increasing it.", UserWarning)
        if momentum_token_pool_size < 100:
            warnings.warn(f"Momentum token pool size ({momentum_token_pool_size}) is small. Consider increasing it.", UserWarning)
        
        # Generate token pools
        self.tokens_pool = [f"INDEX_{i}" for i in range(index_token_pool_size)]
        self.momentum_pool = [f"MOMENTUM_{i}" for i in range(momentum_token_pool_size)]
        
        # Regular expression patterns for token replacement
        self.pattern_momentum = re.compile(r'\b[ijkl]_\d{1,}\b')
        self.pattern_num_123 = re.compile(r'\b(?![ps]_)\w+_\d{1,}\b')
        self.pattern_special = re.compile(r'\b\w+_+\w+\b\\')
        self.pattern_underscore_curly = re.compile(r'\b\w+_{')
        self.pattern_prop = re.compile(r'Prop')
        self.pattern_int = re.compile(r'int\{')
        self.pattern_operators = {
            '+': re.compile(r'\+'), '-': re.compile(r'-'), '*': re.compile(r'\*'),
            ',': re.compile(r','), '^': re.compile(r'\^'), '%': re.compile(r'%'),
            '}': re.compile(r'\}'), '(': re.compile(r'\('), ')': re.compile(r'\)')
        }
        self.pattern_mass = re.compile(r'\b\w+_\w\b')
        self.pattern_s = re.compile(r'\b\w+_\d{2,}\b')
        self.pattern_reg_prop = re.compile(r'\b\w+_\d{1}\b')
        self.pattern_antipart = re.compile(r'(\w)_\w+_\d+\(X\)\^\(\*\)')
        self.pattern_part = re.compile(r'(\w)_\w+_\d+\(X\)')
        self.pattern_index = re.compile(r'\b\w+_\w+_\d{2,}\b')
        
        self.special_symbols = special_symbols
        self.UNK_IDX = UNK_IDX
        self.to_replace = to_replace

    @staticmethod
    def remove_whitespace(expression):
        """Remove all forms of whitespace from the expression."""
        return re.sub(r'\s+', '', expression)

    @staticmethod
    def split_expression(expression):
        """Split the expression by space delimiter."""
        return re.split(r' ', expression)

    def build_tgt_vocab(self):
        """Build vocabulary for target sequences."""
        counter = Counter()
        for eqn in tqdm(self.sqamps, desc='Processing target vocab'):
            counter.update(self.tgt_tokenize(eqn))
        voc = vocab(OrderedDict(counter), specials=self.special_symbols[:], special_first=True)
        voc.set_default_index(self.UNK_IDX)
        return voc

    def build_src_vocab(self, seed):
        """Build vocabulary for source sequences."""
        counter = Counter()
        for diag in tqdm(self.amps, desc='Processing source vocab'):
            counter.update(self.src_tokenize(diag, seed))
        voc = vocab(OrderedDict(counter), specials=self.special_symbols[:], special_first=True)
        voc.set_default_index(self.UNK_IDX)
        return voc
    
    def src_replace(self, ampl, seed):
        """Replace indexed and momentum variables with tokenized equivalents."""
        ampl = self.remove_whitespace(ampl)
        
        random.seed(seed)
        token_cycle = cycle(random.sample(self.tokens_pool, len(self.tokens_pool)))
        momentum_cycle = cycle(random.sample(self.momentum_pool, len(self.momentum_pool)))
        
        # Replace momentum tokens
        temp_ampl = ampl
        momentum_mapping = {match: next(momentum_cycle) for match in set(self.pattern_momentum.findall(ampl))}
        for key, value in momentum_mapping.items():
            temp_ampl = temp_ampl.replace(key, value)
        
        # Replace index tokens
        num_123_mapping = {match: next(token_cycle) for match in set(self.pattern_num_123.findall(ampl))}
        for key, value in num_123_mapping.items():
            temp_ampl = temp_ampl.replace(key, value)

        # Replace pattern index tokens
        pattern_index_mapping = {match: f"{'_'.join(match.split('_')[:-1])} {next(token_cycle)}"
                for match in set(self.pattern_index.findall(ampl))
            }
        for key, value in pattern_index_mapping.items():
            temp_ampl = temp_ampl.replace(key, value)
            
        return temp_ampl
    
    def src_tokenize(self, ampl, seed):
        """Tokenize source expression, optionally applying replacements."""
        temp_ampl = self.src_replace(ampl, seed) if self.to_replace else ampl
        temp_ampl = temp_ampl.replace('\\\\', '\\').replace('\\', ' \\ ').replace('%', '')

        temp_ampl = self.pattern_underscore_curly.sub(lambda match: f' {match.group(0)} ', temp_ampl)

        
        for symbol, pattern in self.pattern_operators.items():
            temp_ampl = pattern.sub(f' {symbol} ', temp_ampl)
        
        temp_ampl = re.sub(r' {2,}', ' ', temp_ampl)
        return [token for token in self.split_expression(temp_ampl) if token]

    def tgt_tokenize(self, sqampl):
        """Tokenize target expression."""
        sqampl = self.remove_whitespace(sqampl)
        temp_sqampl = sqampl
        
        for symbol, pattern in self.pattern_operators.items():
            temp_sqampl = pattern.sub(f' {symbol} ', temp_sqampl)
        
        for pattern in [self.pattern_reg_prop, self.pattern_mass, self.pattern_s]:
            temp_sqampl = pattern.sub(lambda match: f' {match.group(0)} ', temp_sqampl)
        
        temp_sqampl = re.sub(r' {2,}', ' ', temp_sqampl)
        return [token for token in self.split_expression(temp_sqampl) if token]

In [3]:
def read_file_to_list(file_path):
    lines_list = []
    with open(file_path, 'r') as file:
        for line in file:
            lines_list.append(line.strip())  # .strip() removes newline characters
    return lines_list

In [4]:
def normalize_indices(tokenizer, expressions, index_token_pool_size=50, momentum_token_pool_size=50):
    # Function to replace indices with a new set of tokens for each expression
    def replace_indices(token_list, index_map):
        new_index = (f"INDEX_{i}" for i in range(index_token_pool_size))  # Local generator for new indices
        new_tokens = []
        for token in token_list:
            if "INDEX_" in token:
                if token not in index_map:
                    try:
                        index_map[token] = next(new_index)
                    except StopIteration:
                        # Handle the case where no more indices are available
                        raise ValueError("Ran out of unique indices, increase token_pool_size")
                new_tokens.append(index_map[token])
            else:
                new_tokens.append(token)
        return new_tokens

    def replace_momenta(token_list, index_map):
        new_index = (f"MOMENTUM_{i}" for i in range(momentum_token_pool_size))  # Local generator for new indices
        new_tokens = []
        for token in token_list:
            if "MOMENTUM_" in token:
                if token not in index_map:
                    try:
                        index_map[token] = next(new_index)
                    except StopIteration:
                        # Handle the case where no more indices are available
                        raise ValueError("Ran out of unique indices, increase momentum_token_pool_size")
                new_tokens.append(index_map[token])
            else:
                new_tokens.append(token)
        return new_tokens

    normalized_expressions = []
    # Replace indices in each expression randomly
    for expr in tqdm(expressions,desc="Normalizing.."):
        toks = tokenizer.src_tokenize(expr,42)
        normalized_expressions.append(replace_momenta(replace_indices(toks, {}), {}))

    return normalized_expressions

In [5]:
def aug_data(df):
    # Extract columns
    amps = df['amp']
    sqamps = df['sqamp']

    # Data augmentation
    n_samples = 1
    aug_amps = []

    for amp in tqdm(amps, desc='processing'):
        random_seed = [random.randint(1, 1000) for _ in range(n_samples)]
        for seed in random_seed:
            aug_amps.append(tokenizer.src_replace(amp, seed))

    aug_sqamps = [sqamp for sqamp in sqamps for _ in range(n_samples)]


    normal_amps = normalize_indices(tokenizer,aug_amps,INDEX_POOL_SIZE,MOMENTUM_POOL_SIZE)
    
    aug_amps = []
    for amp in normal_amps:
        aug_amps.append("".join(amp))

    # Create augmented DataFrame
    df_aug = pd.DataFrame({"amp": aug_amps, "sqamp": aug_sqamps})

    return df_aug

In [6]:
curr_dir = "/pscratch/sd/r/ritesh11/raw_data/SYMBA_test_data"
line_list = [read_file_to_list(os.path.join(curr_dir, file)) for file in os.listdir(curr_dir)]

In [7]:
import random


data_train = {'sqamp': [], 'process': [], 'amp': []}
data_valid = {'sqamp': [], 'process': [], 'amp': []}
data_test = {'sqamp': [], 'process': [], 'amp': []}

data_list = []


for lines in line_list:
    for c in lines:
        res = c.split(" : ")
        try:
            data_list.append((res[1], res[2], res[3]))
        except IndexError:
            pass
        except Exception:
            if len(res) > 2 and "error" not in res[2]:
                print(res)
                break


random.shuffle(data_list)

total = len(data_list)
train_size = int(0.80 * total)
valid_size = int(0.10 * total)

# Assign data to splits
data_train['process'], data_train['amp'], data_train['sqamp'] = zip(*data_list[:train_size])
data_valid['process'], data_valid['amp'], data_valid['sqamp'] = zip(*data_list[train_size:train_size + valid_size])
data_test['process'], data_test['amp'], data_test['sqamp'] = zip(*data_list[train_size + valid_size:])


In [8]:
# Special token indices
BOS_IDX = 0  # Beginning of Sequence
PAD_IDX = 1  # Padding
EOS_IDX = 2  # End of Sequence
UNK_IDX = 3  # Unknown Token
SEP_IDX = 4  # Separator Token

# Special token symbols
SPECIAL_SYMBOLS = ['<S>', '<PAD>', '</S>', '<UNK>', '<SEP>']

INDEX_POOL_SIZE = 200
MOMENTUM_POOL_SIZE = 200

In [9]:
df_train = pd.DataFrame(data_train)
df_test = pd.DataFrame(data_valid)
df_valid = pd.DataFrame(data_test)

In [28]:
df_train

,sqamp,process,amp
0,16/81*e^4*m_s^2*(m_s^2 + 1/4*s_13)*(m_s^2 + -s...,"Vertex V_0: OffShell A(X_2), OffShell s(X_3),...","1/9*i*e^2*(m_s*gamma_{%\rho_426719,%eta_216031..."
1,1/81*e^4*(16*m_b^2*m_d^2 + 8*m_d^2*s_12 + 8*s_...,"Vertex V_0:AntiPart d(X_3), d(X_4), OffShell ...","-1/9*i*e^2*gamma_{+%\tau_56188,%gam_26119,%eta..."
2,1/9*e^4*(16*m_b^2*m_e^2 + 8*m_b^2*s_12 + 8*s_1...,"Vertex V_1:AntiPart b(X_3), OffShell b(X_4), ...","-1/3*i*e^2*gamma_{+%\tau_82943,%gam_57488,%del..."
3,e^4*(16*m_e^2*m_mu^2 + (-8)*m_mu^2*s_13 + 8*s_...,"Vertex V_1: OffShell mu(X_2), OffShell mu(X_4...","-i*e^2*gamma_{+%\sigma_158360,%eta_49850,%eps_..."
4,1/36*e^4*(16*m_b^2*m_t^2 + 8*m_b^2*s_12 + 8*s_...,"Vertex V_1:AntiPart b(X_3), b(X_4), OffShell ...","-1/6*i*e^2*gamma_{+%\tau_98491,%gam_57008,%del..."
...,...,...,...
12436,2*e^4*s_23*s_24*(s_23 + -1/2*reg_prop)^(-2) + ...,"Vertex V_0:A(X_2), e(X_3), AntiPart OffShell ...","-1/2*i*e^2*(p_2_%\mu_6398*gamma_{+%\mu_6398,%g..."
12437,4/9*e^4*(16*m_e^2*m_tt^2 + 8*m_tt^2*s_12 + 8*s...,"Vertex V_1:AntiPart OffShell tt(X_3), OffShe...","2/3*i*e^2*gamma_{+%\sigma_82763,%gam_47766,%de..."
12438,e^4*(16*m_e^4 + (-8)*m_e^2*s_13 + 8*s_14*s_23 ...,"Vertex V_1:e(X_2), OffShell e(X_4), OffShell...","i*e^2*gamma_{+%\sigma_7873,%gam_10888,%del_741..."
12439,2/81*e^4*s_23*s_24*(s_13 + -1/2*reg_prop)^(-2)...,"Vertex V_0:A(X_2), s(X_4), AntiPart OffShell ...",-1/18*i*e^2*(p_2_%\rho_133929*gamma_{%\rho_133...


In [16]:
tokenizer = Tokenizer(_, 500, 500, _, _, False)

In [30]:
df_train = aug_data(df_train)
df_valid = aug_data(df_valid)
df_test = aug_data(df_test)

Normalizing..: 100%|██████████| 1555/1555 [00:00<00:00, 15381.14it/s]


In [31]:
df_train.shape, df_valid.shape, df_test.shape

((12441, 2), (1556, 2), (1555, 2))

In [32]:
df_train.to_csv("SYMBA_testtrain.csv",index=False)
df_test.to_csv("SYMBA_testtest.csv",index=False)
df_valid.to_csv("SYMBA_testvalid.csv",index=False)

In [17]:
df_train_small = pd.read_csv("EW_2-3train.csv")
df_test_small = pd.read_csv("EW_2-3test.csv")
df_valid_small = pd.read_csv("EW_2-3valid.csv")

In [18]:
pd.concat([df_train,df_train_small]).sample(frac=1).to_csv("EW_2-2-3train.csv",index=False)
pd.concat([df_test,df_test_small]).sample(frac=1).to_csv("EW_2-2-3test.csv",index=False)
pd.concat([df_valid,df_valid_small]).sample(frac=1).to_csv("EW_2-2-3valid.csv",index=False)